In [ ]:
!pip uninstall -y transformers

Found existing installation: transformers 4.30.0
Uninstalling transformers-4.30.0:
  Successfully uninstalled transformers-4.30.0


In [ ]:
!pip install transformers==4.30.0

  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.0 which is incompatible.


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import pandas as pd

In [ ]:
dataset = load_dataset("code_x_glue_cc_code_refinement",'small')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/591k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/589k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/46680 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5835 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5835 [00:00<?, ? examples/s]

In [ ]:
from transformers import RobertaTokenizer
model_name = "Salesforce/codet5-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
def preprocess(example):
    input_text = "fix: " + example['buggy']
    target_text = example['fixed']
    model_input = tokenizer(input_text, max_length=128, padding="max_length", truncation=True)
    label = tokenizer(target_text, max_length=128, padding="max_length", truncation=True)
    model_input["labels"] = label["input_ids"]
    return model_input

In [ ]:
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/46680 [00:00<?, ? examples/s]

Map:   0%|          | 0/5835 [00:00<?, ? examples/s]

Map:   0%|          | 0/5835 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = tokenized_dataset["train"].select(range(1000))
tokenized_eval = tokenized_dataset["validation"].select(range(200))

In [ ]:
training_args = TrainingArguments(
    output_dir="./codet5-fix-model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),
    report_to=None,
    remove_unused_columns=False,
    gradient_accumulation_steps=4,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: viswa80 (viswa80-rajalakshmi-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.048900,0.045558


TrainOutput(global_step=5835, training_loss=0.05978089132774765, metrics={'train_runtime': 3797.7547, 'train_samples_per_second': 12.291, 'train_steps_per_second': 1.536, 'total_flos': 7106538582835200.0, 'train_loss': 0.05978089132774765, 'epoch': 1.0})

In [ ]:
trainer.save_model("./codet5-fix-model")
tokenizer.save_pretrained("./codet5-fix-model")


('./codet5-fix-model/tokenizer_config.json',
 './codet5-fix-model/special_tokens_map.json',
 './codet5-fix-model/vocab.json',
 './codet5-fix-model/merges.txt',
 './codet5-fix-model/added_tokens.json')

In [ ]:
from transformers import T5ForConditionalGeneration, RobertaTokenizer

model_path = "./codet5-fix-model"

tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def fix_code(buggy_code):
    input_text = "fix: " + buggy_code
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fixed_code

buggy = "def add(a,b):\nreturn a+b"
fixed = fix_code(buggy)
print("Fixed Code:\n", fixed)


Fixed Code:
 public static void add ( java.lang.String a , java.lang.String b ) { if ( b == null ) { return a ; } else { a += b ; } } 



In [ ]:
from transformers import T5ForConditionalGeneration, RobertaTokenizer

model_path = "./codet5-fix-model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")

buggy_code = "public static boolean isEmpty(String str) { return str.length() == 0; }"
input_text = "fix: " + buggy_code

inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
outputs = model.generate(
    **inputs,
    max_length=128,
    num_return_sequences=1,
    num_beams=5,
    early_stopping=True
)


fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🔧 Fixed Code:\n" + fixed_code.strip())

🔧 Fixed Code:
public static boolean isEmpty ( java.lang.String str ) { return ( str == null ) || ( str . isEmpty ( ) ) ; }public static boolean isEmpty ( java.lang.String str ) { return str.trim ( ) . isEmpty ( ) ; }


In [ ]:
from google.colab import files

files.download("/content/codet5_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

shutil.make_archive("/content/codet5_model", 'zip', './codet5-fix-model')


'/content/codet5_model.zip'

In [ ]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_cc_code_refinement", "small")

print("Available splits:", dataset.keys())

print("\nExample from train split:")
print(dataset['train'][0])

print("\nExample from validation split:")
print(dataset['validation'][0])

print("\nExample from test split:")
print(dataset['test'][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/591k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/589k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/46680 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5835 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5835 [00:00<?, ? examples/s]

Available splits: dict_keys(['train', 'validation', 'test'])

Example from train split:
{'id': 0, 'buggy': 'public java.lang.String METHOD_1 ( ) { return new TYPE_1 ( STRING_1 ) . format ( VAR_1 [ ( ( VAR_1 . length ) - 1 ) ] . getTime ( ) ) ; } \n', 'fixed': 'public java.lang.String METHOD_1 ( ) { return new TYPE_1 ( STRING_1 ) . format ( VAR_1 [ ( ( type ) - 1 ) ] . getTime ( ) ) ; } \n'}

Example from validation split:
{'id': 0, 'buggy': 'public java.util.List < TYPE_1 > METHOD_1 ( ) { java.util.ArrayList < TYPE_1 > VAR_1 = new java.util.ArrayList < TYPE_1 > ( ) ; for ( TYPE_2 VAR_2 : VAR_3 ) { VAR_1 . METHOD_2 ( VAR_2 . METHOD_1 ( ) ) ; } return VAR_1 ; } \n', 'fixed': 'public java.util.List < TYPE_1 > METHOD_1 ( ) { return VAR_1 ; } \n'}

Example from test split:
{'id': 0, 'buggy': 'private TYPE_1 getType ( TYPE_2 VAR_1 ) { TYPE_3 VAR_2 = new TYPE_3 ( STRING_1 ) ; return new TYPE_1 ( VAR_2 , VAR_2 ) ; } \n', 'fixed': 'private TYPE_1 getType ( TYPE_2 VAR_1 ) { TYPE_3 VAR_2 = new TY

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which